## Camera models with NumPy and Matplotlib
In this script we will experience different camera models and play with then in terms of rotations and translations.\
This script is a condensation of the work done by [Mario Larcher](https://github.com/mnslarcher/camera-models) with some added features done by me.
In order to run the program it's necessary to import the libraries used in the original project.( those can be find in the github)

**Import libraries**

In [1]:
%matplotlib qt
import matplotlib.pyplot as plt
import numpy as np

from camera_models import *  # our package


DECIMALS = 2  # how many decimal places to use in print

%load_ext autoreload
%autoreload 2

### Homogeneous coordinates

In [2]:
X = np.array([4.0, 2.0, 3.0])
print(f"X: {X}")
Xh = to_homogeneus(X)
print(f"X in homogeneous coordinates: {Xh}")

X: [4. 2. 3.]
X in homogeneous coordinates: [4. 2. 3. 1.]


## Camera translation and rotation
Points in space are expressed in term of the **world coordinate frame**. The camera will often be translated and rotated relative to it. The parameters of $\rm R$, the $3 \times 3$ **rotation matrix** representing the orientation of the **camera coordinate frame** and $\boldsymbol{\rm C}$, the coordinates of the camera centre in the world coordinate frame, are called the **external** parameters or the **exterior orientation**.

### Camera translation 
Since we will express the coordinates of the camera with respect to the world coordinate frame (whose origin $\boldsymbol{\rm O}$ is equal to $\boldsymbol{\rm 0} = \begin{pmatrix}0.0 & 0.0 & 0.0\end{pmatrix}$), the translation vector $\boldsymbol{\rm t}$ will be equivalent to the camera center, since $\boldsymbol{\rm C} = \boldsymbol{\rm O} + \boldsymbol{\rm t} = \boldsymbol{\rm 0} + \boldsymbol{\rm t} = \boldsymbol{\rm t}$.

In [3]:
world_origin = np.zeros(3)  # world location(0,0,0)
dx, dy, dz = np.eye(3)      # creates a 1's diagonal
t = np.array([3,-4,2])      # camera translation 

#world frame
world_frame = ReferenceFrame(
    origin=world_origin, 
    dx=dx, 
    dy=dy,
    dz=dz,
    name="World",
)

#camera frame -> 
camera_frame = ReferenceFrame(
    origin=t,
    dx=dx, 
    dy=dy,
    dz=dz,
    name="Camera",
)

# plot
fig = plt.figure()
ax = fig.add_subplot(projection = '3d')
world_frame.draw3d()
camera_frame.draw3d()
draw3d_arrow(world_origin, t, color="tab:red", name="t")        # draw arrow to connect 
set_xyzlim3d(-3, 3)
ax.set_title(f"Camera Translation (t = {t})")
plt.tight_layout()
plt.show()

### Camera Rotation

![title](/home/alegria/camera-models-main/figures/638px-Yaw_Axis_Corrected.svg.png)


$$
\rm R = \rm R_z \rm R_y \rm R_x = 
\begin{bmatrix}
\cos \theta_z & -\sin \theta_z & 0 \\
\sin \theta_z &  \cos \theta_z & 0 \\
 0            &   0            & 1 \\
\end{bmatrix}
\begin{bmatrix}
 \cos \theta_y & 0 & \sin \theta_y \\
  0            & 1 &  0 \\
-\sin \theta_y & 0 & \cos \theta_y \\
\end{bmatrix}
\begin{bmatrix}
1 &  0            &   0 \\
0 & \cos \theta_x & -\sin \theta_x \\
0 & \sin \theta_x &  \cos \theta_x \\
\end{bmatrix}
= 
\begin{bmatrix}
\cos\theta_z\cos\theta_y & \cos\theta_z\sin\theta_y\sin\theta_x - \sin\theta_z\cos\theta_x & \cos\theta_z\sin\theta_y\cos\theta_x + \sin\theta_z\sin\theta_x \\
\sin\theta_z\cos\theta_y & \sin\theta_z\sin\theta_y\sin\theta_x + \cos\theta_z\cos\theta_x & \sin\theta_z\sin\theta_y\cos\theta_x - \cos\theta_z\sin\theta_x \\
-\sin\theta_y & \cos\theta_y\sin\theta_x & \cos\theta_y\cos\theta_x \\
\end{bmatrix}\,.
$$
The angles $\theta_x$, $\theta_y$ and $\theta_z$ are often called **Euler angles** or, more correctly **Tait–Bryan angles**, given that in the original Euler formulation the same axis was considered twice (Z-X-Z, Y-X-Y, etc.). These angles are also associated with the concepts of **roll**, **pitch** and **yaw** (see the image above).

In [4]:
Rx = get_rotation_matrix(theta_x=np.pi / 2.0)
Ry = get_rotation_matrix(theta_y=np.pi / 2.0)
Rz = get_rotation_matrix(theta_z=np.pi / 2.0)
dx, dy, dz = np.eye(3)
world_frame = ReferenceFrame(
    origin=np.zeros(3), 
    dx=dx, 
    dy=dy,
    dz=dz,
    name="World",
)
roll_frame = ReferenceFrame(
    origin=np.zeros(3), 
    dx=Rx @ dx, 
    dy=Rx @ dy,
    dz=Rx @ dz,
    name="Camera",
)
pitch_frame = ReferenceFrame(
    origin=np.zeros(3), 
    dx=Ry @ dx, 
    dy=Ry @ dy,
    dz=Ry @ dz,
    name="Camera",
)
yaw_frame = ReferenceFrame(
    origin=np.zeros(3), 
    dx=Rz @ dx, 
    dy=Rz @ dy,
    dz=Rz @ dz,
    name="Camera",
)
fig = plt.figure()

ax = fig.add_subplot(2, 2, 1, projection="3d")
world_frame.draw3d()
set_xyzlim3d(-1, 1)
set_xyzticks([])
ax.set_title(f"No Rotation")

ax = fig.add_subplot(2, 2, 2, projection="3d")
roll_frame.draw3d()
set_xyzlim3d(-1, 1)
ax.set_title(f"Roll (90°)")
set_xyzticks([])

ax = fig.add_subplot(2, 2, 3, projection="3d")
pitch_frame.draw3d()
set_xyzlim3d(-1, 1)
set_xyzticks([])
ax.set_title(f"Pitch (90°)")

ax = fig.add_subplot(2, 2, 4, projection="3d")
yaw_frame.draw3d()
set_xyzlim3d(-1, 1)
set_xyzticks([])
ax.set_title(f"Yaw (90°)")

fig.suptitle("Camera Rotation")
plt.tight_layout()
plt.show()

## Pinhole Camera Geometry

![title](/home/alegria/calibration/camera_models/pinhole_camera.png)


Let's define some quantities:
* **Focal length** ($f$): distance between the camera centre and the image plane.
* **Principal axis** or **Principal ray** ($\rm Z$) of the camera: line from the camera centre perpendicular to the image plane.
* **Principal point** ($\boldsymbol{\rm p}$): point where the principal axis meets the image plane, its coordinates with respect to the reference system of the image are $(p_x, p_y)^\top$.

We can now visually represent the pinhole camera geometry.

> **Note** - the sensor size was rounded due to some error when projecting the image plane.

In [5]:
F = 3                                       # focal length( in mm )
sensor_size = np.array([11,7])              # sensor size(mm)
PX= sensor_size[0]/2.0                      # principal point x-coordinate
PY= sensor_size[1]/2.0                                     # principal point y-coordinate
THETA_X = np.pi / 2                         # roll angle
THETA_Z = np.pi                             # yaw angle
C = np.array([6, -5, 2])                    # camera centre
IMAGE_HEIGTH = sensor_size[1]
IMAGE_WIDTH = sensor_size[0]

In [6]:
R = get_rotation_matrix(theta_x=THETA_X, theta_z=THETA_Z)
world_origin = np.zeros(3)
dx, dy, dz = np.eye(3)
world_frame = ReferenceFrame(
    origin=world_origin, 
    dx=dx, 
    dy=dy,
    dz=dz,
    name="World",
)
camera_frame = ReferenceFrame(
    origin=C, 
    dx=R @ dx, 
    dy=R @ dy,
    dz=R @ dz,
    name="Camera",
)
Z = PrincipalAxis(
    camera_center=camera_frame.origin,
    camera_dz=camera_frame.dz,
    f=F,
)
image_frame = ReferenceFrame(
    origin=Z.p - camera_frame.dx * PX - camera_frame.dy * PY, 
    dx=R @ dx, 
    dy=R @ dy,
    dz=R @ dz,
    name="Image",
)
image_plane = ImagePlane(
    origin=image_frame.origin, 
    dx=image_frame.dx, 
    dy=image_frame.dy, 
    heigth=IMAGE_HEIGTH,
    width=IMAGE_WIDTH,
)

In [7]:
fig = plt.figure(figsize=(6, 6))
ax = fig.add_subplot(projection="3d")
ax.text(*C, "C")
world_frame.draw3d()
camera_frame.draw3d()
image_frame.draw3d()
Z.draw3d()
image_plane.draw3d()
ax.view_init(elev=30.0, azim=30.0)
ax.set_title("Pinhole Camera Geometry")
ax.set_xlabel("X-axis")
ax.set_ylabel("Y-axis")
ax.set_zlabel("Z-axis")
plt.tight_layout()
plt.show()

## Representing a generic point in 3D
### Intersection Between a Line and a Plane
We now add a generic point $\boldsymbol{\rm X}$ in space and see how it is represented in the image plane.

Let us consider the line joining the camera centre $\boldsymbol{\rm C}$ with the point $\boldsymbol{\rm X}$, both represented in homogeneous coordinates. We can represent this line with a $4 \times 4$ skew-symmetric homogeneous matrix, called its **plücker matrix**, that is defined as

$$
\rm L = \boldsymbol{\rm C}\boldsymbol{\rm X}^\top - \boldsymbol{\rm X}\boldsymbol{\rm C}^\top \,.
$$

We can also find the representation in homogeneous coordinates of the image plane $\boldsymbol \pi$ starting from three points on it. For example we can use the origin of the image frame that we call $\boldsymbol{\rm X_1} = \begin{pmatrix}\boldsymbol{\rm \tilde{X}_1} \\ 1\end{pmatrix}$ and the two points $\boldsymbol{\rm X_2} = \begin{pmatrix}\boldsymbol{\rm \tilde{X}_2} \\ 1\end{pmatrix} = \boldsymbol{\rm X_1} + d_x$ and $\boldsymbol{\rm X_3} = \begin{pmatrix}\boldsymbol{\rm \tilde{X}_3} \\ 1\end{pmatrix} = \boldsymbol{\rm X_1} + d_y$ and apply the formula

$$
\boldsymbol \pi = \begin{bmatrix}
(\boldsymbol{\rm \tilde{X}_1} - \boldsymbol{\rm \tilde{X}_3}) \times (\boldsymbol{\rm \tilde{X}_2} - \boldsymbol{\rm \tilde{X}_3}) \\
-\boldsymbol{\rm \tilde{X}_3}^\top (\boldsymbol{\rm \tilde{X}_1} - \boldsymbol{\rm \tilde{X}_2})
\end{bmatrix} \,.
$$

Now that we have all the ingredients, the intersection between the line $\rm L$ and the plane $\boldsymbol \pi$ is simply

$$
\boldsymbol{x} = \rm L \boldsymbol \pi \,.
$$

In [8]:
X = np.array([6, 2,5])                        # Create a random point
G = GenericPoint(X, name="X")                   
L = get_plucker_matrix(C, X)
X1 = image_frame.origin
X2 = X1 + image_frame.dx
X3 = X1 + image_frame.dy
pi = get_plane_from_three_points(X1, X2, X3)
x = to_inhomogeneus(L @ pi)
print(f"X:\n{X}")
print(f"\nL:\n{L.round(DECIMALS)}")
print(f"\nX1:\n{X1}")
print(f"\nX2:\n{X2}")
print(f"\nX3:\n{X3}")
print(f"\npi:\n{pi.round(DECIMALS)}")
print(f"\nx:\n{x.round(DECIMALS)}")

X:
[6 2 5]

L:
[[  0  42  18   0]
 [-42   0 -29  -7]
 [-18  29   0  -3]
 [  0   7   3   0]]

X1:
[11.5 -2.  -1.5]

X2:
[10.5 -2.  -1.5]

X3:
[11.5 -2.  -0.5]

pi:
[0. 1. 0. 2.]

x:
[ 6.   -2.    3.29]


In [9]:
fig = plt.figure(figsize=(6, 6))
ax = fig.add_subplot(projection="3d")
ax.text(*C, "C")
world_frame.draw3d()
camera_frame.draw3d()
image_frame.draw3d()
Z.draw3d()
image_plane.draw3d()
G.draw3d(pi, C=C)
ax.view_init(elev=30.0, azim=30.0)
ax.set_title(f"Representation of a Generic Point in 3D (X = {X})")
ax.set_xlabel("X-axis")
ax.set_ylabel("Y-axis")
ax.set_zlabel("Z-axis")
plt.tight_layout()
plt.show()

### Camera Calibration Matrix 
We now introduce a new matrix $\rm K$, called the **camera calibration matrix**, that contains what are called the **internal** camera parameters, or **internal orientation** of the camera.

$$
\rm K = \begin{bmatrix}
f & 0        & p_x \\
 0       & f & p_y \\
 0       & 0        & 1 \\
\end{bmatrix}\,.
$$

If we assume that the camera centre coincides with the origin of the world coordinate frame and that there is no rotation, the **image point** $\boldsymbol{\rm x}$ is given by $\boldsymbol{\rm x} = \rm K [I | 0] X$, we call $\rm P =  K [I | 0]$ the **camera projection matrix**.

In [10]:
F = 3                                       # focal length( in mm )
sensor_size = np.array([11,7])              # sensor size(mm)
PX= sensor_size[0]/2.0                      # principal point x-coordinate
PY= sensor_size[1]/2.0                                     # principal point y-coordinate
IMAGE_HEIGTH = sensor_size[1]
IMAGE_WIDTH = sensor_size[0]

In [11]:
X = np.array([6, 2, 5])                        # Create a random point
Xh = to_homogeneus(X)
K = get_calibration_matrix(F, px=PX, py=PY)     # calibration matrix
P = get_projection_matrix(F, px=PX, py=PY)      # projection matrix
xh = P @ Xh                                     # projection matrix * point in space(homo)
x = to_inhomogeneus(xh)
print("\nX:\n", X)
print("\nXh:\n",Xh)
print("\nCalibration matrix (K):\n", K)
print("\nProjection matrix (P):\n", P)
print("\nx:\n", x)


X:
 [6 2 5]

Xh:
 [6 2 5 1]

Calibration matrix (K):
 [[3.  0.  5.5]
 [0.  3.  3.5]
 [0.  0.  1. ]]

Projection matrix (P):
 [[3.  0.  5.5 0. ]
 [0.  3.  3.5 0. ]
 [0.  0.  1.  0. ]]

x:
 [9.1 4.7]


In [12]:
image = Image(heigth=IMAGE_HEIGTH, width=IMAGE_WIDTH)
G = GenericPoint(X, name="X")

In [13]:
fig = plt.figure(figsize=(IMAGE_WIDTH, IMAGE_HEIGTH))
ax = fig.add_subplot()
image.draw()
G.draw(F, px=PX, py=PY)
ax.set_title("Image of the Point X")
ax.set_xlabel("X-axis")
ax.set_ylabel("Y-axis")
plt.tight_layout()
plt.show()